In [5]:
park_code = "GRSM"
url = f"https://irma.nps.gov/Stats/MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&_m=Remote&_r=/NPS.Stats.Reports/Park Specific Reports/Recreation Visitors By Month (1979 - Last Calendar Year)&_15=True&_16=True&_18=True&_19=True&_34=False&_35=False&_39=880px&Park={park_code}"
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
  </title>
  <script src="//code.jquery.com/jquery-1.11.0.min.js" type="text/javascript">
  </script>
  <script src="//code.jquery.com/jquery-migrate-1.2.1.min.js" type="text/javascript">
  </script>
 </head>
 <body>
  <form action="./MvcReportViewer.aspx?_id=7e661565-c923-497d-8d4d-8fc0dcc43632&amp;_m=Remote&amp;_r=%2fNPS.Stats.Reports%2fPark+Specific+Reports%2fRecreation+Visitors+By+Month+(1979+-+Last+Calendar+Year)&amp;_15=True&amp;_16=True&amp;_18=True&amp;_19=True&amp;_34=False&amp;_35=False&amp;_39=880px&amp;Park=GRSM" id="reportForm" method="post">
   <input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="qZtUdNMHIiFXVxm0I2e1WZdfH8uSUGzSFqJzaWLhOmkPwTPo6dM8P4ZreUFEG9O5Jjyr5LjkuZynMU1JvnbbEw61JrEK1XsX7wODGwPQuJ129lIPsk9QbNTGqMLqr89dwjREO+w5/BcHdhG3IoB3GAfqBi8bp8/lctVCJG5D24xpsVFs0miEt54KmmBeilaEvZPh4WTrm5etGSwdZwXsP2DjliqmcKOSocKHpDu4wFz9MJUq+ZcUJHgYZW+NkSQ1yrhikWrAsRSh2Q/Lbk89Kjm/4YhbqAaNn2I/V